### Opis Zadania

Twoim celem jest przetestowanie i przystosowanie do dobrego treningu modelu seq-to-seq, który będzie mógł rozwiązać problem tłumaczenia automatycznego. Model ma nauczyć się na podstawie zbioru danych tekstowych ```pol_neg.csv``` i nauczyć się przewidywać kolejne słowa na podstawie częściowo wprowadzonego zdania.

**Instrukcje**

1. Zaimportuj odpowiednie moduły z biblioteki Keras i pobierz dane do treningu.

2. Uruchom architekturę modelu seq-to-seq z wykorzystaniem warstw LSTM. Model  ma enkoder i dekoder, a także warstwę gęstą na końcu dekodera.

3. Skonfiguruj model i wybierz optymalizator learning rate oraz funkcję straty. Dodaj liczbę epoch i odpowiedni batch size.

4. Trenuj model na  danych treningowych.

6. Oceń wyniki treningu i przetestuj działanie modelu.

Zapisz wytrenowany model do pliku, aby mógł być używany w przyszłości.

In [19]:
import pandas as pd

df = pd.read_csv('/kaggle/input/ankiweb-polish-english/pol_eng.csv')
df.head()

input_texts = df['angielski']
target_texts = df['polski']
target_texts=target_texts.str.replace('\n','')
target_texts=target_texts.str.replace('\t','')
df['polski']=target_texts
# Utwórz zbiór unikalnych znaków dla polskiego i angielskiego
input_characters = set().union(*df['angielski'].apply(set))
target_characters = set().union(*df['polski'].apply(set))


num_encoder_tokens = len(sorted(list(input_characters)))
num_decoder_tokens = len( sorted(list(target_characters)))
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input (PL) tokens:', num_encoder_tokens)
print('Number of unique output (EN) tokens:', num_decoder_tokens)
print('Max sequence length for inputs (PL):', max_encoder_seq_length)
print('Max sequence length for outputs (EN):', max_decoder_seq_length)


Number of samples: 10000
Number of unique input (PL) tokens: 71
Number of unique output (EN) tokens: 86
Max sequence length for inputs (PL): 19
Max sequence length for outputs (EN): 56


In [12]:
target_texts=target_texts.str.replace('\n','')
target_texts=target_texts.str.replace('\t','')


In [16]:
df['polski']

0                        Idź.
1                      Cześć.
2                    Uciekaj!
3                    Biegnij.
4                    Uciekaj.
                ...          
9995     Tom zmienił pociągi.
9996      Tom przekonał Mary.
9997    Tom gotuje dla Marii.
9998    Tom mógłby wam pomóc.
9999     Tom mógłby ci pomóc.
Name: polski, Length: 10000, dtype: object

In [3]:
!wget https://github.com/marcin119a/PUMP2/raw/main/decoder_input_data.npz & wget https://github.com/marcin119a/PUMP2/raw/main/decoder_target_data.npz & wget https://github.com/marcin119a/PUMP2/raw/main/encoder_input_data.npz

--2024-02-13 17:17:36--  https://github.com/marcin119a/PUMP2/raw/main/decoder_target_data.npz
--2024-02-13 17:17:36--  https://github.com/marcin119a/PUMP2/raw/main/decoder_input_data.npz
--2024-02-13 17:17:36--  https://github.com/marcin119a/PUMP2/raw/main/encoder_input_data.npz
Resolving github.com (github.com)... Resolving github.com (github.com)... Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
connected.
connected.
HTTP request sent, awaiting response... HTTP request sent, awaiting response... HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/marcin119a/PUMP2/main/decoder_target_data.npz [following]
--2024-02-13 17:17:37--  https://raw.githubusercontent.com/marcin119a/PUMP2/main/decoder_target_data.npz
Resolving raw.githubuse

In [17]:
import numpy as np
# Wczytujemy arrayy ze skompresowanego pliku .npz
decoder_input_data = np.load('decoder_input_data.npz')['decoder_input_data']
decoder_target_data = np.load('decoder_target_data.npz')['my_array']
encoder_input_data = np.load('encoder_input_data.npz')['encoder_input_data']

### TODO trening modelu do wysokiego accuracy

In [18]:
latent_dim = 512
batch_size = 1024
epochs = 60

from tensorflow.keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input, LSTM, Dense

# Zdefiniuj sekwencję wejściową i przetwórz ją.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# Odrzucamy `encoder_outputs` i zachowujemy tylko stany.
encoder_states = [state_h, state_c]

# Skonfiguruj dekoder, używając `encoder_states` jako stanu początkowego.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# Ustawiamy dekoder tak, aby zwracał pełne sekwencje wyjściowe,
# oraz zwracał również stany wewnętrzne. Nie używamy stanów w modelu treningowym,
# ale wykorzystamy je w predykcji.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Zdefiniuj model, który zamieni `encoder_input_data` i `decoder_input_data` w `decoder_target_data`.
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Uruchom trening
model.compile(optimizer=Adam(learning_rate=0.004), loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Zapisz model
model.save('s2s.h5')


Epoch 1/60


ValueError: in user code:

    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "/opt/conda/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 1 of layer "model" i

In [8]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

# Kolejny krok: tryb wnioskowania (sampling).
# Oto kroki do wykonania:
# 1) Zakoduj dane wejściowe i pobierz początkowy stan dekodera.
# 2) Uruchom jeden krok dekodera z tym początkowym stanem
#    i tokenem "początek sekwencji" jako cel.
#    Wynikiem będzie następny token celu.
# 3) Powtórz z bieżącym tokenem celu i bieżącymi stanami.


# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)


decoder_states_inputs = [Input(shape=(latent_dim,)), Input(shape=(latent_dim,))]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# indeks tokena wyszukiwania wstecznego do dekodowania sekwencji z powrotem do
# coś czytelnego.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Zakoduj dane wejściowe w postaci wektorów stanu.
    states_value = encoder_model.predict(input_seq)

    # Wygeneruj pustą sekwencję celu o długości 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Wypełnij pierwszy znak sekwencji celu znakiem początkowym.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Pętla losowania dla jednej sekwencji (dla uproszczenia, zakładamy rozmiar batcha równy 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value, verbose=0)

        # Wylosuj token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Warunek zakończenia pętli: osiągnięcie maksymalnej długości sekwencji
        # lub znalezienie znaku zakończenia.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Aktualizuj sekwencję celu (o długości 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Aktualizuj stany
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(10):
    # Wybierz jedną sekwencję (część zestawu treningowego)
    # do wypróbowania dekodowania.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)


1/1 [==============================] - 0s 489ms/step
-
Input sentence: Go.
Decoded sentence: wśvFśfFŹfFŹdńn:Ź56FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
1/1 [==============================] - 0s 25ms/step
-
Input sentence: Hi.
Decoded sentence: s:ń:FŹf56FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
1/1 [==============================] - 0s 23ms/step
-
Input sentence: Run!
Decoded sentence: BHĆ

1/1 [==============================] - 0s 23ms/step


KeyboardInterrupt: 

### Zadanie: Klasyfikacja języka na podstawie pojedynczych liter

Twoim zadaniem jest stworzenie modelu sieci neuronowej, który będzie w stanie klasyfikować teksty w różnych językach na podstawie pojedynczych liter. W zbiorze danych znajdują się teksty w języku angielskim i polskim, a Twoim celem jest nauczenie modelu rozpoznawania, do którego języka należy dany tekst.

Kroki, które musisz podjąć:

1. **Przygotowanie danych**: Dane tekstowe zostały już wczytane z pliku CSV. W zbiorze danych znajdują się teksty w języku angielskim i polskim. Każdy tekst został podzielony na pojedyncze litery, a litery zostały przekształcone na sekwencje liczb. Twoim zadaniem jest podzielenie danych na zbiór uczący (training) i zbiór testowy (test).

2. **Model sieci neuronowej**: Zdefiniuj model sieci rekurencyjnej. Jest to model oparty na warstwach Embedding, LSTM i GlobalMaxPooling1D, z warstwami Dense na końcu. Model ma za zadanie klasyfikować teksty na podstawie pojedynczych liter.

3. **Kompilacja modelu**: Model musi zostać skompilowany z odpowiednim optymalizatorem ('adam') i funkcją straty ('binary_crossentropy'). Ponieważ mamy problem binarnej klasyfikacji języka, używamy funkcji 'binary_crossentropy'.

4. **Trenowanie modelu**: Trenuj model na danych treningowych. W kodzie już jest zaimplementowane trenowanie na jednej epoce (epochs=1), ale możesz dostosować to do swoich potrzeb. Warto eksperymentować z różnymi liczbami epok i innymi parametrami trenowania.

5. **Ewaluacja modelu**: Po zakończeniu trenowania, ocen model na danych testowych. Sprawdź, jak dobrze model klasyfikuje teksty na podstawie pojedynczych liter, i oblicz dokładność klasyfikacji (accuracy).

Twoim celem jest stworzenie modelu, który będzie w stanie skutecznie rozpoznawać język na podstawie pojedynczych liter. Możesz eksperymentować z architekturą modelu, parametrami trenowania i innymi aspektami, aby osiągnąć jak najlepsze wyniki.


In [22]:
import pandas as pd 

df = pd.read_csv('/kaggle/input/ankiweb-polish-english/pol_eng.csv')

In [23]:
target_texts = df['polski']
target_texts=target_texts.str.replace('\n','')
target_texts=target_texts.str.replace('\t','')
df['polski']=target_texts

In [24]:
df

,Unnamed: 0,angielski,polski
0,0,Go.,Idź.
1,1,Hi.,Cześć.
2,2,Run!,Uciekaj!
3,3,Run.,Biegnij.
4,4,Run.,Uciekaj.
...,...,...,...
9995,9995,Tom changed trains.,Tom zmienił pociągi.
9996,9996,Tom convinced Mary.,Tom przekonał Mary.
9997,9997,Tom cooks for Mary.,Tom gotuje dla Marii.
9998,9998,Tom could help you.,Tom mógłby wam pomóc.


In [25]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

# Przygotowanie danych próbkowych (zbiór tekstów w różnych językach)
texts = df['angielski'].to_list() + df['polski'].to_list()

# Pojedyncze litery
letter_texts = [' '.join(list(text)) for text in texts]

labels = np.repeat(np.arange(2), 10000)

# Tokenizacja i konwersja tekstów na sekwencje liczb
tokenizer = Tokenizer(char_level=True)  # Ustawienie char_level na True
tokenizer.fit_on_texts(letter_texts)
sequences = tokenizer.texts_to_sequences(letter_texts)

# Wypełnienie sekwencji do tej samej długości
max_len = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_len)

# Teraz sequences zawiera ztokenizowane litery.


In [27]:
X_train, X_test, y_train, y_test = train_test_split(sequences, labels, test_size=0.2, random_state=42)


In [21]:
labels

array([0, 0, 0, ..., 1, 1, 1])

In [35]:
#todo model
model=Sequential()
model.add(Embedding(input_dim=115,output_dim=115))
model.add(LSTM(115))
#model.add(GlobalMaxPooling1D())
#model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.05), metrics=['accuracy'])
model.fit(X_train,y_train, epochs=10, batch_size=1024, validation_split=0.2)


Epoch 1/10
13/13 [==============================] - 4s 170ms/step - loss: 0.6523 - accuracy: 0.6311 - val_loss: 0.3805 - val_accuracy: 0.8409
Epoch 2/10
13/13 [==============================] - 1s 89ms/step - loss: 0.2178 - accuracy: 0.9120 - val_loss: 0.0938 - val_accuracy: 0.9706
Epoch 3/10
13/13 [==============================] - 1s 55ms/step - loss: 0.0699 - accuracy: 0.9793 - val_loss: 0.0402 - val_accuracy: 0.9853
Epoch 4/10
13/13 [==============================] - 1s 43ms/step - loss: 0.0367 - accuracy: 0.9873 - val_loss: 0.0282 - val_accuracy: 0.9916
Epoch 5/10
13/13 [==============================] - 1s 59ms/step - loss: 0.0280 - accuracy: 0.9904 - val_loss: 0.0222 - val_accuracy: 0.9928
Epoch 6/10
13/13 [==============================] - 1s 43ms/step - loss: 0.0221 - accuracy: 0.9918 - val_loss: 0.0226 - val_accuracy: 0.9928
Epoch 7/10
13/13 [==============================] - 1s 55ms/step - loss: 0.0121 - accuracy: 0.9959 - val_loss: 0.0143 - val_accuracy: 0.9959
Epoch 8/10
1

In [36]:
model.evaluate(X_test, y_test, verbose=0)

[0.021389856934547424, 0.9925000071525574]


---

### Zadanie: Detekcja języka tekstu

Twoim zadaniem jest stworzenie modelu, który będzie w stanie rozpoznawać język tekstu na podstawie dostępnych danych. W zbiorze danych znajdują się teksty w różnych językach, a Twoim celem jest nauczenie modelu identyfikować język każdego tekstu.

Kroki, które musisz podjąć:

1. **Przygotowanie danych**: Dane tekstowe zostały już wczytane z pliku CSV. W zbiorze danych znajdują się teksty w różnych językach. Każdy tekst został podzielony na sekwencje słów, a słowa zostały przekształcone na sekwencje liczb. Twoim zadaniem jest podzielenie danych na zbiór uczący (training) i zbiór testowy (test).

2. **Mapowanie języków**: Przygotuj mapowanie języków na unikalne identyfikatory, które będzie można użyć w modelu. Mapowanie zostało już zaimplementowane w kodzie.

3. **Tokenizacja tekstu**: Przekształć tekst na sekwencje liczbowe za pomocą tokenizacji. Ustaw stałą długość sekwencji (np. przez dodawanie zer na końcu sekwencji).

4. **Model sieci neuronowej**: Przygoutj model sieci neuronowej. Jest to model oparty na warstwach Embedding, LSTM i Dense. Model ma za zadanie klasyfikować teksty na podstawie języka.

5. **Kompilacja modelu**: Model musi zostać skompilowany z odpowiednim optymalizatorem ('adam') i funkcją straty ('categorical_crossentropy'). Ponieważ mamy problem wieloklasowej klasyfikacji języka, używamy funkcji straty 'categorical_crossentropy'.

6. **Trenowanie modelu**: Trenuj model na danych treningowych. Możesz dostosować liczbę epok, rozmiar wsadu (batch size) i inne parametry trenowania, aby uzyskać jak najlepsze wyniki.

7. **Ewaluacja modelu**: Po zakończeniu trenowania, ocen model na danych testowych. Sprawdź, jak dobrze model identyfikuje język tekstów, i oblicz dokładność klasyfikacji (accuracy).

Twoim celem jest stworzenie modelu, który będzie w stanie skutecznie rozpoznawać język tekstu na podstawie dostępnych danych. Możesz eksperymentować z architekturą modelu, parametrami trenowania i innymi aspektami, aby osiągnąć jak najlepsze wyniki.


In [37]:
import pandas as pd 

df = pd.read_csv('/kaggle/input/language-detection/Language Detection.csv')

In [38]:
import pandas as pd
from sklearn.model_selection import train_test_split

texts = df['Text'].values
languages = df['Language'].values

# Przygotuj mapowanie języków na unikalne identyfikatory
language_to_id = {lang: i for i, lang in enumerate(set(languages))}
id_to_language = {i: lang for lang, i in language_to_id.items()}
num_languages = len(language_to_id)


In [48]:
language_to_id


{'Malayalam': 0,
 'Portugeese': 1,
 'Italian': 2,
 'French': 3,
 'Dutch': 4,
 'Turkish': 5,
 'Sweedish': 6,
 'Kannada': 7,
 'Russian': 8,
 'Danish': 9,
 'Tamil': 10,
 'Greek': 11,
 'English': 12,
 'German': 13,
 'Hindi': 14,
 'Arabic': 15,
 'Spanish': 16}

In [39]:
#todo tokenizer i model + pad_sequences
# Tokenizacja i konwersja tekstów na sekwencje liczb

from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.utils import pad_sequences
import numpy as np
from tensorflow.keras.optimizers import Adam

# Tokenizacja tekstu
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Ustawienie stałej długości sekwencji
max_len = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_len)

# Podział danych na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(sequences, languages, test_size=0.2, random_state=42)

# Przygotowanie etykiet w postaci kategorialnej
y_train = to_categorical([language_to_id[lang] for lang in y_train], num_classes=num_languages)
y_test = to_categorical([language_to_id[lang] for lang in y_test], num_classes=num_languages)

# Definicja modelu
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=64, input_length=max_len))
model.add(LSTM(64, return_sequences=True))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dense(num_languages, activation='softmax'))

# Kompilacja modelu
model.compile(optimizer=Adam(learning_rate=0.05), loss='categorical_crossentropy', metrics=['accuracy'])

# Trenowanie modelu
model.fit(X_train, y_train, epochs=2, batch_size=128, validation_split=0.1)

# Ewaluacja modelu
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)



Epoch 1/2
59/59 [==============================] - 46s 727ms/step - loss: 0.9850 - accuracy: 0.6733 - val_loss: 0.5484 - val_accuracy: 0.8174
Epoch 2/2
65/65 [==============================] - 14s 209ms/step - loss: 0.3613 - accuracy: 0.8772
Test Accuracy: 0.8771759867668152


In [49]:
#tokenizer.texts_to_sequences(['test'])

model.predict(pad_sequences(tokenizer.texts_to_sequences(['humans']),maxlen=max_len))

1/1 [==============================] - 0s 223ms/step


array([[7.6992037e-05, 2.4113026e-02, 1.1043198e-02, 3.3211742e-02,
        2.6673719e-01, 1.2736706e-01, 8.3265781e-02, 2.3529463e-05,
        1.0524538e-03, 6.8296693e-02, 2.2484134e-03, 2.9869105e-03,
        8.9769691e-02, 2.4984522e-01, 3.3598193e-03, 2.9332782e-03,
        3.3669081e-02]], dtype=float32)